In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "xxxx"
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

os.environ["OPENAI_API_KEY"] = "xxxx"

os.environ["TAVILY_API_KEY"] = "xxxx"

In [2]:
!pip install numpy==1.26.4

In [3]:
import numpy as np

print(np.__version__)
assert np.__version__ == "1.26.4"

1.26.4


In [4]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 \
    langchain-community==0.3.0 GitPython==3.1.43 \
    langchain-chroma==0.1.4 tavily-python==0.5.0 pydantic==2.10.6

## ドキュメントの読み込み

In [10]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".md")


loader = GitLoader(
    clone_url="https://github.com/open-mmlab/mmsegmentation",
    repo_path="./mmsegmentation",
    branch="main",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

172


## ドキュメントを適切なサイズにチャンク化

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

split_docs = text_splitter.split_documents(documents)
print(f"チャンク数: {len(split_docs)}")

チャンク数: 2420


## 文書をベクトル化

In [21]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(split_docs, embeddings)


## シンプルなRAG

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke("mmsegmentationの概要を教えて")

'mmsegmentationは、画像セグメンテーションのためのオープンソースライブラリであり、深層学習を用いたセグメンテーション手法を提供します。このライブラリは、さまざまなセグメンテーションモデルをサポートしており、ユーザーが簡単にモデルを訓練、評価、推論できるように設計されています。ドキュメントには、インストール手順や使用方法、FAQなどが含まれており、ユーザーがスムーズに利用を開始できるようにサポートしています。具体的な内容については、[こちらのリンク](https://mmsegmentation.readthedocs.io/en/latest/)から詳細なドキュメントを参照できます。'

## HyDE
LLMに仮説的な回答をさせ、それに類似したベクトルを抽出する

In [23]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。

質問: {question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

In [24]:
hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke("mmsegmentationの概要を教えて")

'MMSegmentationは、PyTorchに基づいたオープンソースのセマンティックセグメンテーションツールボックスです。OpenMMLabプロジェクトの一部であり、セマンティックセグメンテーション手法の統一された実装と評価のためのフレームワークを提供します。このツールボックスには、人気のあるセマンティックセグメンテーション手法やデータセットの高品質な実装が含まれています。\n\nMMSegmentationは、以下の7つの主要部分で構成されています：\n\n1. **apis**: モデル推論のための高レベルAPIを提供。\n2. **structures**: セグメンテーションデータ構造`SegDataSample`を提供。\n3. **datasets**: セマンティックセグメンテーション用のさまざまなデータセットをサポート。\n   - **transforms**: 有用なデータ拡張変換を多数含む。\n4. **models**: セグメンターの最も重要な部分で、セグメンターのさまざまなコンポーネントを含む。\n   - **segmentors**: すべてのセグメントモデルクラスを定義。\n   - **data_preprocessors**: モデルの入力データを前処理するためのもの。\n   - **backbones**: 画像を特徴マップにマッピングするさまざまなバックボーンネットワークを含む。\n   - **necks**: セグメンテーションヘッドとバックボーンネットワークを接続するためのさまざまなモデルネックコンポーネントを含む。\n   - **decode_heads**: 特徴マップを入力として受け取り、セグメンテーション結果を予測するさまざまなセグメンテーションヘッドを含む。\n   - **losses**: さまざまな損失関数を含む。\n\nMMSegmentationは、セマンティックセグメンテーションの研究や実装を行うための強力なツールです。'

### 複数の仮説的な回答を元にベクトル探索をする

In [25]:
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリスト")


query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文書を検索するために、
3つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標です。

質問: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

In [26]:
multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("mmsegmentationの概要を教えて")

'MMSegmentationは、統一された実装と評価のためのフレームワークを提供するツールボックスで、主にセマンティックセグメンテーションの手法を対象としています。このツールボックスには、人気のあるセマンティックセグメンテーション手法やデータセットの高品質な実装が含まれています。\n\nMMSegmentationは、以下の7つの主要な部分で構成されています：\n\n1. **apis**: モデル推論のための高レベルAPIを提供。\n2. **structures**: セグメンテーションデータ構造`SegDataSample`を提供。\n3. **datasets**: セマンティックセグメンテーション用のさまざまなデータセットをサポート。\n   - **transforms**: 有用なデータ拡張変換を多数含む。\n4. **models**: セグメンターにとって最も重要な部分で、セグメンターのさまざまなコンポーネントを含む。\n5. **engine**: モデルのトレーニングや評価を行うエンジン。\n6. **evaluation**: モデルの評価を行うための機能。\n7. **visualization**: セグメンテーション結果を視覚化するためのツール。\n\nMMSegmentationは、PyTorchに基づいており、OpenMMLabプロジェクトの一部として開発されています。'